# Electricity Generation from api.electricitymap.org
---

SECTION 1 
collect data on carbon intensity, zones within SW , and power breakdown from the electricitymap api

7-23   Still problematic.

In [196]:
# Dependencies and Setup

import pandas as pd
import requests


In [197]:

# zones for electrical utilities in US
zones = ["US-SW-PNM", "US-SW-EPE", "US-SW-WALC", "US-NW-PACE", "US-NW-PSCO", "US-CENT-SWPP", "US-TEX-ERCO", "US-MIDW-AECI","US-SW-AZPS","US-SW-AZPS",
         "US-NW-WACM", "US-SW-SRP", "US-SW-TEPC", "US-CENT-SPA", "US-CAL-IID", "US-CAL-CISO", "US-CAL-BANC","US-CAL-BANC", "US-CAL-TIDC", 
          "US-CAR-CPLE", "US-CAR-CPLW", "US-CAR-DUK", "US-CAR-SC", "US-CAR-SCEG", "US-CAR-YAD", "US-FLA-FMPP", "US-FLA-FPC" , "US-FLA-FPL",
          "US-FLA-GVL" , "US-FLA-HST", "US-FLA-JEA", "US-FLA-SEC", "US-FLA-TAL", "US-FLA-TEC", "US-MIDW-AECI" , "US-MIDW-LGEE", "US-MIDW-MISO",
          "US-NE-ISNE", "US-NW-BPAT", "US-NW-CHPD", "US-NW-DOPD", "US-NW-GCPD", "US-NW-GRID",  "US-NW-IPCO" , "US-NW-NWMT", "US-NW-NEVP", 
           "US-NW-PACW",  "US-NW-PGE", "US-NW-PSEI", "US-NW-SCL", "US-NW-TPWR", "US-NW-WAUW", "US-NY-NYIS", "US-SE-SEPA", "US-SE-SOCO" , 
           "US-TEN-TVA"]


In [198]:

# get carbon intensity history for the US utilities
urls = []
for index, url in enumerate(zones):
    url = f'https://api.electricitymap.org/v3/carbon-intensity/history?zone={zones[index]}'
    urls.append(url)

# store responses in responses_dict dictionary
responses_dict = {}
for idx, url in enumerate(urls):
    response = requests.get(url)
    responses_dict[f"response_{idx+1}"] = response.json()


In [199]:
#request power breakdown
pburls = []
for index, url in enumerate(zones):
    pburl = f'https://api.electricitymap.org/v3/power-breakdown/history?zone={zones[index]}'
    pburls.append(pburl)

#store response in power_breakdown_responses_dict dictionary
power_breakdown_responses_dict = {}
for idx, pburl in enumerate(pburls):
    response = requests.get(pburl)
    power_breakdown_responses_dict[f"response_{idx+1}"] = response.json()


SECTION 2     
PowerBreakdown data transformation

In [200]:
# pull data from power breakdown response and the carbon_intensity ressponse
region = power_breakdown_responses_dict['response_1']['history'][0]['zone']
datetime = power_breakdown_responses_dict['response_1']['history'][0]['datetime']
nuclear = power_breakdown_responses_dict['response_1']['history'][0]["powerConsumptionBreakdown"]['nuclear']
geothermal = power_breakdown_responses_dict['response_1']['history'][0]["powerConsumptionBreakdown"]['geothermal']
biomass = power_breakdown_responses_dict['response_1']['history'][0]["powerConsumptionBreakdown"]['biomass']
coal = power_breakdown_responses_dict['response_1']['history'][0]["powerConsumptionBreakdown"]['coal']
wind = power_breakdown_responses_dict['response_1']['history'][0]["powerConsumptionBreakdown"]['wind']
solar = power_breakdown_responses_dict['response_1']['history'][0]["powerConsumptionBreakdown"]['solar']
hydro = power_breakdown_responses_dict['response_1']['history'][0]["powerConsumptionBreakdown"]['hydro']
gas = power_breakdown_responses_dict['response_1']['history'][0]["powerConsumptionBreakdown"]['gas']
oil = power_breakdown_responses_dict['response_1']['history'][0]["powerConsumptionBreakdown"]['oil']
unknown = power_breakdown_responses_dict['response_1']['history'][0]["powerConsumptionBreakdown"]['unknown']
hydro_discharge = power_breakdown_responses_dict['response_1']['history'][0]["powerConsumptionBreakdown"]['hydro discharge']
battery_discharge = power_breakdown_responses_dict['response_1']['history'][0]["powerConsumptionBreakdown"]['battery discharge']
renewable_percentage = power_breakdown_responses_dict['response_1']['history'][0]["renewablePercentage"]
total_consumption = power_breakdown_responses_dict['response_1']['history'][0]["powerConsumptionTotal"]
estimated = power_breakdown_responses_dict['response_1']['history'][0]["isEstimated"]
carbon_Intensity = responses_dict['response_1']['history'][0]["carbonIntensity"]
estimated_C = responses_dict['response_1']['history'][0]["isEstimated"]

# create a dictionary with first values for this zone
us_pnm1 = {'region':region,'datetime':datetime,'nuclear':nuclear,'geothermal':geothermal,'biomass':biomass, 'coal':coal, 'wind':wind, 'solar':solar, 
           'hydro':hydro, 'gas':gas, 'oil':oil, 'unknown':unknown, 'hydro-discharge':hydro_discharge, 
           'battery_discharge':battery_discharge, 'renewable_percentage':renewable_percentage, 'total_consumption':total_consumption, 
           'estimated':estimated, 'carbon_Intensity':carbon_Intensity, 'estimated_C':estimated_C}

# Create a dataFrame with the first values
df_US = pd.DataFrame.from_dict(us_pnm1,orient='index')


In [213]:
# Data wrangling from the response to create a legible dataFrame
# outer for loop for regions/responses
for reg in range(len(zones)):
    #for each zone
    response = f"response_{reg+1}"
   
# pull data from response for each time in this file for this region and add to the existing dataframe
    for i in range(24):
        # 24 is for the 24 hours of data for each zone
        region = power_breakdown_responses_dict[response]['history'][i]['zone']
        datetime = power_breakdown_responses_dict[response]['history'][i]['datetime']
        nuclear = power_breakdown_responses_dict[response]['history'][i]["powerConsumptionBreakdown"]['nuclear']
        geothermal = power_breakdown_responses_dict[response]['history'][i]["powerConsumptionBreakdown"]['geothermal']
        biomass = power_breakdown_responses_dict[response]['history'][i]["powerConsumptionBreakdown"]['biomass']
        coal = power_breakdown_responses_dict[response]['history'][i]["powerConsumptionBreakdown"]['coal']
        wind = power_breakdown_responses_dict[response]['history'][i]["powerConsumptionBreakdown"]['wind']
        solar = power_breakdown_responses_dict[response]['history'][i]["powerConsumptionBreakdown"]['solar']
        hydro = power_breakdown_responses_dict[response]['history'][i]["powerConsumptionBreakdown"]['hydro']
        gas = power_breakdown_responses_dict[response]['history'][i]["powerConsumptionBreakdown"]['gas']
        oil = power_breakdown_responses_dict[response]['history'][i]["powerConsumptionBreakdown"]['oil']
        unknown = power_breakdown_responses_dict[response]['history'][i]["powerConsumptionBreakdown"]['unknown']
        hydro_discharge = power_breakdown_responses_dict[response]['history'][i]["powerConsumptionBreakdown"]['hydro discharge']
        battery_discharge = power_breakdown_responses_dict[response]['history'][i]["powerConsumptionBreakdown"]['battery discharge']
        renewable_percentage = power_breakdown_responses_dict[response]['history'][i]["renewablePercentage"]
        total_consumption = power_breakdown_responses_dict[response]['history'][i]["powerConsumptionTotal"]
        estimated = power_breakdown_responses_dict[response]['history'][i]["isEstimated"]
        carbon_Intensity = responses_dict[response]['history'][i]["carbonIntensity"]
        estimated_C = responses_dict[response]['history'][i]["isEstimated"]

        # this 24 is also for the 23 hours of data for each zone
        df_US[24*reg+i]= {'region':region, 'datetime':datetime,'nuclear':nuclear,'geothermal':geothermal,'biomass':biomass, 'coal':coal, 'wind':wind, 'solar':solar, 
           'hydro':hydro, 'gas':gas, 'oil':oil, 'unknown':unknown, 'hydro-discharge':hydro_discharge, 
           'battery_discharge':battery_discharge, 'renewable_percentage':renewable_percentage, 'total_consumption':total_consumption, 
           'estimated':estimated, 'carbon_Intensity':carbon_Intensity, 'estimated_C':estimated_C}
#set up the times as rows and measurements as columns
df_US_new = df_US.transpose()

# check data types
df_US_new.describe()

,region,datetime,nuclear,geothermal,biomass,coal,wind,solar,hydro,gas,oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption,estimated,carbon_Intensity,estimated_C
count,1344,1344,1344,1344,1344,1344,1344,1344,1344,1344,1344,1344,1344,1344,1333,1344,1344,1333,1344
unique,53,24,287,24,55,568,212,326,564,928,45,271,1,30,83,1133,2,543,2
top,US-CAL-BANC,2024-07-22T20:00:00.000Z,0,0,0,0,0,0,0,0,0,0,0,0,0,0,True,24,True
freq,48,56,822,1320,1210,608,949,819,331,192,1203,626,1344,1305,253,11,1230,181,1230


In [221]:
# fill NA values with zeroes for energy values
df_US_new = df_US_new.fillna({'nuclear': 0,'geothermal': 0,'biomass': 0, 'coal': 0, 'wind': 0, 'solar': 0, 'carbon_Intensity':0,
           'hydro': 0, 'gas': 0, 'oil': 0, 'unknown': 0, 'hydro-discharge':0, 'renewable_percentage':0, 'total_consumption':0,
           'battery_discharge':0})
df_US_new.describe()

,nuclear,geothermal,biomass,coal,wind,solar,hydro,gas,oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption,carbon_Intensity
count,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.0,1344.000000,1344.000000,1344.000000,1344.000000
mean,1152.497024,12.522321,19.062500,1636.375000,103.330357,454.981399,580.142857,4133.715774,1.867560,86.965030,0.0,17.881696,27.580357,8199.359375,452.325893
std,2498.461077,93.060078,97.312665,4125.351369,448.068087,1963.646831,1224.493075,8261.642288,8.426743,196.394768,0.0,265.953459,34.172064,15443.290583,259.675155
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,276.750000,0.000000,0.000000,0.0,0.000000,1.000000,785.750000,300.750000
50%,0.000000,0.000000,0.000000,28.500000,0.000000,0.000000,58.500000,1086.000000,0.000000,2.000000,0.0,0.000000,12.000000,2352.000000,450.000000
75%,406.000000,0.000000,0.000000,1402.250000,3.000000,66.500000,633.750000,2563.000000,0.000000,58.000000,0.0,0.000000,44.000000,7524.250000,640.000000
max,13566.000000,747.000000,602.000000,29904.000000,5312.000000,16973.000000,9292.000000,48604.000000,116.000000,1596.000000,0.0,5601.000000,100.000000,98801.000000,985.000000


In [222]:
# convert measured Energy values to integers in Giga Watts
#convert_dict = {'renewable_percentage': int}
convert_dict = {'nuclear': int, 'geothermal': int, 'biomass': int, 'coal': int, 'wind': int, 'solar': int, 'hydro': int, 'gas': int, 'oil': int, 
                'unknown':int, 'hydro-discharge': int, 'battery_discharge': int, 'renewable_percentage': int, 'total_consumption': int, 'carbon_Intensity': int
                }
 # note - the unknown column only has values rarely - converting null values to integer doesn't work so this is left as an object
df_US_new = df_US_new.astype(convert_dict)

#check that data types are changed to int
df_US_new.dtypes

region                  object
datetime                object
nuclear                  int32
geothermal               int32
biomass                  int32
coal                     int32
wind                     int32
solar                    int32
hydro                    int32
gas                      int32
oil                      int32
unknown                  int32
hydro-discharge          int32
battery_discharge        int32
renewable_percentage     int32
total_consumption        int32
estimated               object
carbon_Intensity         int32
estimated_C             object
dtype: object

In [223]:
# Date Time work

# import datetime dependencies

from datetime import datetime

# set up lists to hold parsed data and DateTime as a datetime datetype
dates=[]
times = []
DateTime =[]

# convert date time strings
for i in range(len(df_US_new['datetime'])):

    # Parse the timestamp string to a datetime object
    dt_obj = datetime.strptime(df_US_new.iloc[i,1], '%Y-%m-%dT%H:%M:%S.%fZ')

    date = dt_obj.strftime('%Y-%m-%d')
    time = dt_obj.strftime('%H:%M:%S')

#add the new times and dates to lists

    dates.append(date)
    times.append(time)
    DateTime.append(dt_obj)

# add the times and dates to new columns in the data frame
df_US_new['UTC time'] = times
df_US_new['UTC date'] = dates
df_US_new['UTC DateTime'] = DateTime

In [224]:
#set the UTC DateTime as the index
df_US_new_reindex = df_US_new.set_index('UTC DateTime', inplace=True)

#drop the datetime column that contains a string
df_US_newer = df_US_new.drop('datetime', axis=1)
df_US_newer.head()

,region,nuclear,geothermal,biomass,coal,wind,solar,hydro,gas,oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption,estimated,carbon_Intensity,estimated_C,UTC time,UTC date
UTC DateTime,,,,,,,,,,,,,,,,,,,,
2024-07-22 20:00:00,US-SW-PNM,0,0,0,339,0,0,18,472,0,0,0,0,2,829,True,817,True,20:00:00,2024-07-22
2024-07-22 21:00:00,US-SW-PNM,0,0,0,350,0,0,18,514,0,0,0,0,2,882,True,808,True,21:00:00,2024-07-22
2024-07-22 22:00:00,US-SW-PNM,0,0,0,389,0,0,18,553,0,0,0,0,2,959,True,816,True,22:00:00,2024-07-22
2024-07-22 23:00:00,US-SW-PNM,0,0,0,431,0,0,18,585,0,1,0,0,2,1035,True,825,True,23:00:00,2024-07-22
2024-07-23 00:00:00,US-SW-PNM,0,0,0,452,0,0,18,600,0,22,0,0,2,1092,True,827,True,00:00:00,2024-07-23


In [225]:
# convert carbon intensity measurement to an integer in g CO2e/kWh
convert_dict_C= {'carbon_Intensity': int}
 
df_US_newer = df_US_newer.astype(convert_dict_C)

#check that the datatype has been changed
df_US_newer.dtypes

region                  object
nuclear                  int32
geothermal               int32
biomass                  int32
coal                     int32
wind                     int32
solar                    int32
hydro                    int32
gas                      int32
oil                      int32
unknown                  int32
hydro-discharge          int32
battery_discharge        int32
renewable_percentage     int32
total_consumption        int32
estimated               object
carbon_Intensity         int32
estimated_C             object
UTC time                object
UTC date                object
dtype: object

Section 3
Merge new dataframe with running data from previous dates in runningUSenergy_data.csv

In [226]:
# import previous cleaned file into a pandas dataframe
df_us_energy = pd.read_csv('data/runningUSenergy_data.csv')
df_us_energy_reindex=df_us_energy.set_index("UTC DateTime")
df_us_energy_reindex.describe()

C:\Users\mrkol\AppData\Local\Temp\ipykernel_37404\2571967791.py:2: DtypeWarning: Columns (16,20,30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df_us_energy = pd.read_csv('data/runningUSenergy_data.csv')


,nuclear(GW),geothermal(GW),biomass(GW),coal(GW),wind(GW),solar(GW),hydro(GW),gas(GW),oil,unknown,...,nuclear,geothermal,biomass,coal,wind,solar,hydro,gas,total_consumption,carbon_Intensity
count,17206.000000,17206.000000,17206.000000,17206.000000,17206.000000,17206.000000,17206.000000,17206.000000,17254.000000,17254.000000,...,48.0,48.0,48.0,48.000000,48.000000,48.000000,48.0,48.000000,48.000000,48.000000
mean,1221.815297,14.627339,22.863536,1704.047483,498.162443,507.030861,634.424736,4183.559049,5.437058,94.060913,...,0.0,0.0,0.0,351.833333,196.125000,423.166667,18.0,533.125000,1552.104167,358.500000
std,2559.429172,103.168036,107.552201,4470.661436,1874.223990,2043.599352,1444.126307,8220.348857,46.633388,199.243454,...,0.0,0.0,0.0,88.284437,336.617507,443.893668,0.0,127.405973,485.022888,148.050308
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,157.000000,0.000000,0.000000,18.0,310.000000,904.000000,212.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,231.000000,0.000000,0.000000,...,0.0,0.0,0.0,304.000000,0.000000,0.000000,18.0,422.250000,1191.500000,212.000000
50%,0.000000,0.000000,0.000000,22.000000,0.000000,0.000000,73.000000,964.000000,0.000000,8.000000,...,0.0,0.0,0.0,340.500000,21.500000,206.000000,18.0,546.500000,1534.500000,358.500000
75%,650.000000,0.000000,0.000000,1351.000000,135.000000,149.000000,675.000000,3320.000000,0.000000,90.000000,...,0.0,0.0,0.0,388.000000,239.750000,925.500000,18.0,643.000000,1761.250000,505.000000
max,13803.000000,812.000000,657.000000,49357.000000,21642.000000,19768.000000,65867.000000,54713.000000,2054.000000,2053.000000,...,0.0,0.0,0.0,502.000000,1325.000000,1055.000000,18.0,718.000000,2701.000000,505.000000


In [227]:
df_us_energy_reindex.drop_duplicates(inplace=True)
df_us_energy_reindex.describe()

,nuclear(GW),geothermal(GW),biomass(GW),coal(GW),wind(GW),solar(GW),hydro(GW),gas(GW),oil,unknown,...,nuclear,geothermal,biomass,coal,wind,solar,hydro,gas,total_consumption,carbon_Intensity
count,17206.000000,17206.000000,17206.000000,17206.000000,17206.000000,17206.000000,17206.000000,17206.000000,17254.000000,17254.000000,...,48.0,48.0,48.0,48.000000,48.000000,48.000000,48.0,48.000000,48.000000,48.000000
mean,1221.815297,14.627339,22.863536,1704.047483,498.162443,507.030861,634.424736,4183.559049,5.437058,94.060913,...,0.0,0.0,0.0,351.833333,196.125000,423.166667,18.0,533.125000,1552.104167,358.500000
std,2559.429172,103.168036,107.552201,4470.661436,1874.223990,2043.599352,1444.126307,8220.348857,46.633388,199.243454,...,0.0,0.0,0.0,88.284437,336.617507,443.893668,0.0,127.405973,485.022888,148.050308
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,157.000000,0.000000,0.000000,18.0,310.000000,904.000000,212.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,231.000000,0.000000,0.000000,...,0.0,0.0,0.0,304.000000,0.000000,0.000000,18.0,422.250000,1191.500000,212.000000
50%,0.000000,0.000000,0.000000,22.000000,0.000000,0.000000,73.000000,964.000000,0.000000,8.000000,...,0.0,0.0,0.0,340.500000,21.500000,206.000000,18.0,546.500000,1534.500000,358.500000
75%,650.000000,0.000000,0.000000,1351.000000,135.000000,149.000000,675.000000,3320.000000,0.000000,90.000000,...,0.0,0.0,0.0,388.000000,239.750000,925.500000,18.0,643.000000,1761.250000,505.000000
max,13803.000000,812.000000,657.000000,49357.000000,21642.000000,19768.000000,65867.000000,54713.000000,2054.000000,2053.000000,...,0.0,0.0,0.0,502.000000,1325.000000,1055.000000,18.0,718.000000,2701.000000,505.000000


In [228]:
# concatentate current data with existing file
df_both = pd.concat([df_us_energy_reindex,df_US_newer])

# drop duplicate rows
df_both.drop_duplicates(inplace=True)
df_both.describe()

,nuclear(GW),geothermal(GW),biomass(GW),coal(GW),wind(GW),solar(GW),hydro(GW),gas(GW),oil,unknown,...,nuclear,geothermal,biomass,coal,wind,solar,hydro,gas,total_consumption,carbon_Intensity
count,17206.000000,17206.000000,17206.000000,17206.000000,17206.000000,17206.000000,17206.000000,17206.000000,18526.000000,18526.000000,...,1320.000000,1320.000000,1320.000000,1320.000000,1320.000000,1320.000000,1320.000000,1320.000000,1320.000000,1320.000000
mean,1221.815297,14.627339,22.863536,1704.047483,498.162443,507.030861,634.424736,4183.559049,5.199234,93.798122,...,1173.045455,12.750000,19.409091,1619.302273,111.846212,468.552273,572.665152,4157.354545,8243.081061,441.684091
std,2559.429172,103.168036,107.552201,4470.661436,1874.223990,2043.599352,1444.126307,8220.348857,45.069415,199.350241,...,2516.391364,93.887434,98.159720,4163.429685,456.286075,1980.839838,1233.322326,8338.367355,15586.418983,257.110407
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,231.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,265.000000,769.500000,287.000000
50%,0.000000,0.000000,0.000000,22.000000,0.000000,0.000000,73.000000,964.000000,0.000000,8.000000,...,0.000000,0.000000,0.000000,76.000000,0.000000,0.000000,55.000000,901.000000,2068.500000,446.000000
75%,650.000000,0.000000,0.000000,1351.000000,135.000000,149.000000,675.000000,3320.000000,0.000000,89.000000,...,649.000000,0.000000,0.000000,1159.500000,4.000000,72.250000,539.250000,2790.000000,7798.500000,615.000000
max,13803.000000,812.000000,657.000000,49357.000000,21642.000000,19768.000000,65867.000000,54713.000000,2054.000000,2053.000000,...,13566.000000,747.000000,602.000000,29904.000000,5312.000000,16973.000000,9292.000000,48604.000000,98801.000000,985.000000


In [229]:

df_both.to_csv(f'data/runningUSenergy_datacheck.csv')

In [230]:
df_both_cleaned = df_both.loc[df_both['breakdown estimated?']==False,:]
df_both_cleaned.describe()

,nuclear(GW),geothermal(GW),biomass(GW),coal(GW),wind(GW),solar(GW),hydro(GW),gas(GW),oil,unknown,...,nuclear,geothermal,biomass,coal,wind,solar,hydro,gas,total_consumption,carbon_Intensity
count,5177.000000,5177.000000,5177.000000,5177.000000,5177.000000,5177.000000,5177.000000,5177.000000,5177.000000,5177.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,1411.096388,46.770910,70.738652,1933.457408,928.983002,724.617925,1082.134441,5525.683214,12.725710,114.387869,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,2297.917304,180.392566,179.853837,4651.526840,2550.839814,2720.523579,1889.789247,8286.399297,76.746595,223.567662,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.000000,264.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,0.000000,0.000000,0.000000,26.000000,29.000000,1.000000,223.000000,1473.000000,0.000000,13.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,2245.000000,0.000000,8.000000,1191.000000,413.000000,198.000000,1338.000000,9030.000000,0.000000,127.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,13287.000000,812.000000,657.000000,48997.000000,21642.000000,19768.000000,65867.000000,54713.000000,2054.000000,2053.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [231]:
df_both_cleaned.to_csv(f'data/runningUSenergy_data_filteredcheck.csv')